# Task 4: Portfolio Optimization

In this task, we optimize a portfolio of **TSLA**, **BND**, and **SPY**. We use the TSLA forecast from Task 3 and historical data for bonds and the broad market.

## Step 1: Prepare Expected Returns

- **TSLA:** Calculated from the 12-month future forecast.
- **BND & SPY:** Calculated as annualized historical average daily returns.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pmdarima as pm

# 1. Load historical data
data_path = "../data/processed"
assets = ["TSLA", "BND", "SPY"]
dfs = {asset: pd.read_csv(os.path.join(data_path, f"{asset}_final_processed.csv"), index_col='Date', parse_dates=True) for asset in assets}

# 2. Calculate TSLA expected return from forecast
tsla_hist = dfs['TSLA']['Close']
model_future = pm.auto_arima(tsla_hist, seasonal=False, error_action='ignore', suppress_warnings=True)
future_forecast = model_future.predict(n_periods=252)

last_price = tsla_hist.iloc[-1]
expected_price = future_forecast.iloc[-1]  # Using .iloc for positional indexing
tsla_expected_return = (expected_price / last_price) - 1

print(f"TSLA Expected Annual Return (Forecast-based): {tsla_expected_return:.2%}")

# 3. Calculate BND and SPY historical annualized returns
expected_returns = pd.Series(index=assets, dtype=float)
expected_returns['TSLA'] = tsla_expected_return

for asset in ["BND", "SPY"]:
    daily_returns = dfs[asset]['Close'].pct_change().dropna()
    # Annualize: (1 + avg_daily_return)^252 - 1
    avg_daily = daily_returns.mean()
    annualized_return = (1 + avg_daily)**252 - 1
    expected_returns[asset] = annualized_return
    print(f"{asset} Expected Annual Return (Historical): {annualized_return:.2%}")

print("\n--- Final Expected Returns (mu) ---")
print(expected_returns)

## Step 2: Compute Covariance Matrix

We calculate the daily returns for all assets and then compute the sample covariance matrix.

In [ ]:
from pypfopt import risk_models

# 1. Combine prices into a single DataFrame
prices = pd.concat({ticker: df['Close'] for ticker, df in dfs.items()}, axis=1)
prices = prices.fillna(method='ffill').dropna()

# 2. Calculate Daily Returns
returns = prices.pct_change().dropna()
print(f"Daily returns calculated. Shape: {returns.shape}")

# 3. Compute Sample Covariance Matrix (S)
S = risk_models.sample_cov(prices)

print("\n--- Sample Covariance Matrix (S) ---")
print(S)

## Step 3: Generate the Efficient Frontier

We use `PyPortfolioOpt` to simulate portfolios and calculate the Efficient Frontier based on our parameters ($mu$ and $S$).

In [ ]:
from pypfopt import EfficientFrontier

# 1. Initialize Efficient Frontier
ef = EfficientFrontier(expected_returns, S)

# 2. Optimize for Maximum Sharpe Ratio
raw_weights_max_sharpe = ef.max_sharpe()
weights_max_sharpe = ef.clean_weights()
perf_max_sharpe = ef.portfolio_performance(verbose=True)

# 3. Optimize for Minimum Volatility
ef_min = EfficientFrontier(expected_returns, S)
raw_weights_min_vol = ef_min.min_volatility()
weights_min_vol = ef_min.clean_weights()
perf_min_vol = ef_min.portfolio_performance(verbose=True)

print("\n--- Optimized Weights (Max Sharpe) ---")
for asset, w in weights_max_sharpe.items():
    print(f"{asset}: {w:.4f}")

print("\n--- Optimized Weights (Min Volatility) ---")
for asset, w in weights_min_vol.items():
    print(f"{asset}: {w:.4f}")

## Step 4: Visualize and Mark Key Portfolios

We plot the Efficient Frontier and mark our key portfolios: **Max Sharpe Ratio** and **Min Volatility**.

In [ ]:
from pypfopt import plotting

def plot_efficiency(mu, S, weights_max_sharpe, weights_min_vol):
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Plot the Efficient Frontier
    ef = EfficientFrontier(mu, S)
    plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
    
    # Generate random portfolios to show the feasible region
    n_samples = 5000
    w = np.random.dirichlet(np.ones(len(mu)), n_samples)
    rets = w.dot(mu)
    stds = np.sqrt(np.diag(w @ S @ w.T))
    sharpes = rets / stds
    ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r", alpha=0.3)
    
    # Mark Max Sharpe Ratio Portfolio
    ax.scatter(perf_max_sharpe[1], perf_max_sharpe[0], marker="*", s=200, c="red", label="Max Sharpe Ratio")
    
    # Mark Min Volatility Portfolio
    ax.scatter(perf_min_vol[1], perf_min_vol[0], marker="*", s=200, c="blue", label="Min Volatility")
    
    ax.set_title("Efficient Frontier: TSLA, BND, SPY")
    ax.set_xlabel("Volatility (Risk)")
    ax.set_ylabel("Expected Return")
    ax.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_efficiency(expected_returns, S, weights_max_sharpe, weights_min_vol)

## Step 5: Recommend and Document

In [ ]:
from IPython.display import display, Markdown

# 1. Extract values for the report
w_tsla = weights_max_sharpe['TSLA']
w_bnd = weights_max_sharpe['BND']
w_spy = weights_max_sharpe['SPY']
ann_ret, ann_vol, sharpe = perf_max_sharpe

# 2. Generate the report
report = f"""
### Final Portfolio Recommendation
Based on the analysis, we recommend the **Maximum Sharpe Ratio Portfolio** for investors seeking a balanced growth strategy.

#### **Optimal Weights**
- **TSLA:** {w_tsla:.2%}
- **BND:** {w_bnd:.2%}
- **SPY:** {w_spy:.2%}

#### **Expected Performance Metrics**
- **Expected Annual Return:** {ann_ret:.2%}
- **Annual Volatility:** {ann_vol:.2%}
- **Sharpe Ratio:** {sharpe:.2f}

### **Rationale**
1. **Risk-Adjusted Return:** The Maximum Sharpe Ratio portfolio provides the highest return per unit of risk. By incorporating TSLA (high growth/high risk) with BND (low risk/low return) and SPY (diversified growth), this allocation optimizes the reward-to-variability ratio.
2. **Diversification:** The portfolio leverages the negative or low correlation between bonds (BND) and growth stocks (TSLA/SPY) to flatten the volatility curve compared to a 100% equity position.
3. **Forward-Looking Alignment:** Unlike purely historical models, this recommendation uses the stable 12-month forecast for TSLA, ensuring the allocation reflects current market momentum rather than deprecated price action.

**Conclusion:** For a mid-to-long-term investment horizon, the Max Sharpe allocation offers a robust path to capital appreciation while maintaining a defensive bond buffer against TSLA's inherent volatility.
"""

display(Markdown(report))